In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

In [ ]:
input_names = list(train_data.columns)
input_names.remove('term_deposit_subscribed')

all_inputs = train_data[input_names].values
all_labels = train_data['term_deposit_subscribed'].values

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

obj_cols = train_data.select_dtypes(include='object').columns


In [ ]:
for col in obj_cols:
    train_data[col] = le.fit_transform(train_data[col])

train_data.head(10)


In [ ]:
from sklearn.model_selection import train_test_split

(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(all_inputs_encoded.values, all_labels, test_size=0.25, random_state=1)

In [ ]:
train_data = pd.read_csv('train-clean.csv',delimiter=",")
train_data